In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazonreviews/test.ft.txt.bz2
/kaggle/input/amazonreviews/train.ft.txt.bz2


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
import re
from tensorflow.keras.layers import Embedding, Dense, Dropout, LSTM, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils

from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

2025-05-05 08:27:17.330563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746433637.534312      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746433637.592385      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def process_data(file):
    data = []
    for index, row in file.iterrows():
         # first line data is raw data
        line = row[0]
        
        #split lines into text and labels
        label, text = line.split(' ', 1) 
        
        #remove the __label__ only keep the number
        label = label.replace('__label__', '') 
        
        #append
        data.append((label, text.strip()))
        
    cols = ['label', 'review']
    return pd.DataFrame(data, columns=cols)

In [4]:
def text_cleaning(text):
    
    #apply lowercase
    text = text.lower()  
    
    #remove special characters and numbers and extra whitespace
    pattern_punc = r'[^A-Za-z\s]'
    text = re.sub(pattern_punc, '', text).strip()
    return text

In [5]:
train_path = '/kaggle/input/amazonreviews/train.ft.txt.bz2'
test_path = '/kaggle/input/amazonreviews/test.ft.txt.bz2'

In [6]:
train_data = pd.read_csv(train_path, compression='bz2', delimiter='\t')
test_data = pd.read_csv(test_path, compression='bz2', delimiter='\t')
print(train_data.head())

  __label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
0  __label__2 The best soundtrack ever to anythin...                                                                                                                                                                                                                                                                                                                                                                                                   
1  __label__2 Amazing!: This soundtrack is my fav...                                                                    

In [7]:
print("Train data shape" ,train_data.shape)
print("Test data shape" ,test_data.shape)

Train data shape (3599999, 1)
Test data shape (399999, 1)


In [8]:
train = process_data(train_data)
train.head()

,label,review
0,2,The best soundtrack ever to anything.: I'm rea...
1,2,Amazing!: This soundtrack is my favorite music...
2,2,Excellent Soundtrack: I truly like this soundt...
3,2,"Remember, Pull Your Jaw Off The Floor After He..."
4,2,an absolute masterpiece: I am quite sure any o...


In [9]:
test = process_data(test_data)
test.head()

,label,review
0,2,One of the best game music soundtracks - for a...
1,1,Batteries died within a year ...: I bought thi...
2,2,"works fine, but Maha Energy is better: Check o..."
3,2,Great for the non-audiophile: Reviewed quite a...
4,1,DVD Player crapped out after one year: I also ...


In [10]:
train['label'].value_counts()

label
1    1800000
2    1799999
Name: count, dtype: int64

In [11]:
train['label'] = train['label'].replace({"2":"1", "1":"0"}) #train data
test['label'] = test['label'].replace({"2":"1", "1":"0"}) #test data

In [12]:
print("Count of labels in the train data is:" ,train['label'].value_counts())
print("-"*45)
print("Count of labels in the test data is:" ,test['label'].value_counts())

Count of labels in the train data is: label
0    1800000
1    1799999
Name: count, dtype: int64
---------------------------------------------
Count of labels in the test data is: label
0    200000
1    199999
Name: count, dtype: int64


In [13]:
import re


In [14]:
train['review_cleaned'] = train['review'].apply(text_cleaning)
train.head()

,label,review,review_cleaned
0,1,The best soundtrack ever to anything.: I'm rea...,the best soundtrack ever to anything im readin...
1,1,Amazing!: This soundtrack is my favorite music...,amazing this soundtrack is my favorite music o...
2,1,Excellent Soundtrack: I truly like this soundt...,excellent soundtrack i truly like this soundtr...
3,1,"Remember, Pull Your Jaw Off The Floor After He...",remember pull your jaw off the floor after hea...
4,1,an absolute masterpiece: I am quite sure any o...,an absolute masterpiece i am quite sure any of...


In [15]:
test['review_cleaned'] = test['review'].apply(text_cleaning)
test.head()

,label,review,review_cleaned
0,1,One of the best game music soundtracks - for a...,one of the best game music soundtracks for a ...
1,0,Batteries died within a year ...: I bought thi...,batteries died within a year i bought this ch...
2,1,"works fine, but Maha Energy is better: Check o...",works fine but maha energy is better check out...
3,1,Great for the non-audiophile: Reviewed quite a...,great for the nonaudiophile reviewed quite a b...
4,0,DVD Player crapped out after one year: I also ...,dvd player crapped out after one year i also b...


In [16]:
print(stopwords.words("english"))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [17]:
max_words = 1000
max_len = 100

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(train['review_cleaned'])

X_train = tokenizer.texts_to_sequences(train['review_cleaned'])
X_test = tokenizer.texts_to_sequences(test['review_cleaned'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [18]:
y_train = train['label']
y_test = test['label']

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [20]:
X_train = np.array(X_train)
print("Train:", X_train.shape)
y_train = np.array(y_train)
print("Train:", y_train.shape)
print("-"*50)
X_valid = np.array(X_valid)
print("Valid:", X_valid.shape)
y_valid = np.array(y_valid)
print("Valid:", y_valid.shape)
print("-"*50)
X_test = np.array(X_test)
print("Test:", X_test.shape)
y_test = np.array(y_test)
print("Test:", y_test.shape)

Train: (2879999, 100)
Train: (2879999,)
--------------------------------------------------
Valid: (720000, 100)
Valid: (720000,)
--------------------------------------------------
Test: (399999, 100)
Test: (399999,)


In [21]:
y_train = y_train.astype('int')
y_valid = y_valid.astype('int')
y_test = y_test.astype('int')

In [22]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt


In [23]:
max_words = 10000  # Vocabulary size (adjust based on your dataset)
max_len = 100      # Maximum sequence length (adjust based on your dataset)
epochs = 1
learning_rate = 0.001
batch_size = 32

In [27]:
model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(units=128, return_sequences=True, activation='tanh'))  # Added tanh activation
model.add(Dropout(0.2))  
model.add(LSTM(units=256, activation='tanh'))  # Added tanh activation
model.add(Dense(1, activation='sigmoid'))
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Show model summary
model.summary()

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping],
    verbose=1
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 100, 128)            │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,806,081 (6.89 MB)

 Trainable params: 1,806,081 (6.89 MB)

 Non-trainable params: 0 (0.00 B)

90000/90000 ━━━━━━━━━━━━━━━━━━━━ 1147s 13ms/step - accuracy: 0.8994 - loss: 0.2431 - val_accuracy: 0.9263 - val_loss: 0.1859


In [29]:
def evaluate_model(model, X_test, y_test):
    # Generate predictions
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")
    
    # Display classification report for more details
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return accuracy, precision, recall, f1

# Make sure X_train and y_test have the same number of samples
# If you meant to evaluate on the test set:
accuracy, precision, recall, f1 = evaluate_model(model, X_test, y_test)



12500/12500 ━━━━━━━━━━━━━━━━━━━━ 45s 4ms/step
Test Accuracy: 0.9258
Test Precision: 0.9130
Test Recall: 0.9413
Test F1 Score: 0.9269

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.91      0.92    200000
           1       0.91      0.94      0.93    199999

    accuracy                           0.93    399999
   macro avg       0.93      0.93      0.93    399999
weighted avg       0.93      0.93      0.93    399999



In [30]:
model.save('lstm_text_classification_model.h5')
print("Model saved to lstm_text_classification_model.h5")

Model saved to lstm_text_classification_model.h5


In [22]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt

In [23]:
max_words = 10000  # Vocabulary size (adjust based on your dataset)
max_len = 100      # Maximum sequence length (adjust based on your dataset)
epochs = 1
learning_rate = 0.001
batch_size = 32

In [24]:
model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, activation='tanh')))
model.add(Dropout(0.2))  
model.add(Bidirectional(LSTM(units=128, activation='tanh')))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision', 'Recall']
)

# Show model summary
model.summary()

# Add callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_bidirectional_lstm_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Training
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)


I0000 00:00:1746434635.395232      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746434635.395943      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 100, 256)            │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,953,921 (7.45 MB)

 Trainable params: 1,953,921 (7.45 MB)

 Non-trainable params: 0 (0.00 B)

I0000 00:00:1746434644.848520     107 cuda_dnn.cc:529] Loaded cuDNN version 90300


89999/90000 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - Precision: 0.9014 - Recall: 0.8974 - accuracy: 0.8996 - loss: 0.2417
Epoch 1: val_loss improved from inf to 0.18076, saving model to best_bidirectional_lstm_model.keras
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 1906s 21ms/step - Precision: 0.9014 - Recall: 0.8974 - accuracy: 0.8996 - loss: 0.2417 - val_Precision: 0.9254 - val_Recall: 0.9322 - val_accuracy: 0.9287 - val_loss: 0.1808


In [25]:
def evaluate_model(model, X_test, y_test):
    # Generate predictions
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")
    
    # Display classification report for more details
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return accuracy, precision, recall, f1

# Make sure X_train and y_test have the same number of samples
# If you meant to evaluate on the test set:
accuracy, precision, recall, f1 = evaluate_model(model, X_test, y_test)



12500/12500 ━━━━━━━━━━━━━━━━━━━━ 78s 6ms/step
Test Accuracy: 0.9279
Test Precision: 0.9242
Test Recall: 0.9323
Test F1 Score: 0.9283

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.92      0.93    200000
           1       0.92      0.93      0.93    199999

    accuracy                           0.93    399999
   macro avg       0.93      0.93      0.93    399999
weighted avg       0.93      0.93      0.93    399999



In [26]:
model.save('bidirectional-lstm_text_classification_model.h5')
print("Model saved to bidirectional-lstm_text_classification_model.h5")

Model saved to bidirectional-lstm_text_classification_model.h5


In [27]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, GRU, Dropout, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [28]:
max_words = 10000  # Vocabulary size (adjust based on your dataset)
max_len = 100      # Maximum sequence length (adjust based on your dataset)
epochs = 1
learning_rate = 0.001
batch_size = 32

In [ ]:
model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Bidirectional(GRU(units=128, return_sequences=True, activation='tanh')))
model.add(Dropout(0.2))  
model.add(Bidirectional(GRU(units=128, activation='tanh')))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision', 'Recall']
)

# Show model summary
model.summary()

# Add callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_bidirectional_gru_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Training
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)